In [ ]:
from collections import OrderedDict
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
# XML files were downloaded from:
# https://projects.propublica.org/nonprofits/organizations/990272261
# Note there is more data, but it would involve typing by hand.

with open('2009.xml', 'r') as f: xml2009 = f.read()
with open('2010.xml', 'r') as f: xml2010 = f.read()
with open('2011.xml', 'r') as f: xml2011 = f.read()    
with open('2012.xml', 'r') as f: xml2012 = f.read()    
with open('2013.xml', 'r') as f: xml2013 = f.read()
with open('2014.xml', 'r') as f: xml2014 = f.read()    
with open('2015.xml', 'r') as f: xml2015 = f.read()
with open('2016.xml', 'r') as f: xml2016 = f.read()
with open('2017.xml', 'r') as f: xml2017 = f.read()
with open('2018.xml', 'r') as f: xml2018 = f.read()
with open('2019.xml', 'r') as f: xml2019 = f.read()    
with open('2020.xml', 'r') as f: xml2020 = f.read()

In [ ]:
# It's bad form. But for now, we remove the namespace to make
# the files more readable.  Create element trees.

ns = 'xmlns="http://www.irs.gov/efile"'
return2009 = ET.fromstring(xml2009.replace(ns, ''))
return2010 = ET.fromstring(xml2010.replace(ns, ''))
return2011 = ET.fromstring(xml2011.replace(ns, ''))
return2012 = ET.fromstring(xml2012.replace(ns, ''))
return2013 = ET.fromstring(xml2013.replace(ns, ''))
return2014 = ET.fromstring(xml2014.replace(ns, ''))
return2015 = ET.fromstring(xml2015.replace(ns, ''))
return2016 = ET.fromstring(xml2016.replace(ns, ''))
return2017 = ET.fromstring(xml2017.replace(ns, ''))
return2018 = ET.fromstring(xml2018.replace(ns, ''))
return2019 = ET.fromstring(xml2019.replace(ns, ''))
return2020 = ET.fromstring(xml2020.replace(ns, ''))

In [ ]:
# The IRS appears to have changed nameing convvensions after 2012.
# When reading data, there are two group of files to consider.

returns = OrderedDict()
returns['2009-2012'] = [return2009, return2010, return2011, return2012]
returns['2013-2020'] = [return2013, return2014, return2015, return2016, 
    return2017, return2018, return2019, return2020]

In [ ]:
# Tried lots of things. But this is our current system for dealing with the
# double naming convention.  A goal is to avoid redundant code and errors.

root_tags = OrderedDict()
root_tags['Year'] = {
    '2009-2012': 'ReturnHeader/TaxYear',
    '2013-2020': 'ReturnHeader/TaxYr'}
root_tags['People'] = {
    '2009-2012': 'ReturnData/IRS990/Form990PartVIISectionA',
    '2013-2020': 'ReturnData/IRS990/Form990PartVIISectionAGrp'}

In [ ]:
# Let us see all 'People' tags.  We don't want to miss any.

for era, roots in returns.items():
    print(era)
    tags = []
    for root in roots:
        for person in root.findall(root_tags['People'][era]):
            for child in person:
                tags += [child.tag]
    print(np.unique(tags))

In [ ]:
person_tags = OrderedDict()
person_tags['Name'] =  {
    '2009-2012': 'NamePerson',
    '2013-2020': 'PersonNm'}
person_tags['Title'] = {
    '2009-2012': 'Title',
    '2013-2020': 'TitleTxt'}
person_tags['Director'] = {
    '2009-2012': 'IndividualTrusteeOrDirector',
    '2013-2020': 'IndividualTrusteeOrDirectorInd'}
person_tags['Officer'] = {
    '2009-2012': 'Officer',
    '2013-2020': 'OfficerInd'}
person_tags['Highest'] = {
    '2009-2012': 'HighestCompensatedEmployee',
    '2013-2020': 'HighestCompensatedEmployeeInd'}
person_tags['Former'] = {
    '2009-2012': 'Former',
    '2013-2020': 'FormerOfcrDirectorTrusteeInd'}
person_tags['Pay1'] = {
    '2009-2012': 'ReportableCompFromOrganization',
    '2013-2020': 'ReportableCompFromOrgAmt'}
person_tags['Pay2'] = {
    '2009-2012': 'ReportableCompFromRelatedOrgs',
    '2013-2020': 'ReportableCompFromRltdOrgAmt'}
person_tags['Pay3'] = {
    '2009-2012': 'OtherCompensation',
    '2013-2020': 'OtherCompensationAmt'}
person_tags['Hours1'] = {
    '2009-2012': 'AverageHoursPerWeek',
    '2013-2020': 'AverageHoursPerWeekRt'}
person_tags['Hours2'] = {
    '2009-2012': 'AverageHoursPerWeekRelated',
    '2013-2020': 'AverageHoursPerWeekRltdOrgRt'}

# display(person_tags)

In [ ]:
list0 = []
for era, roots in returns.items():
    for root in roots:
        year = root.find(root_tags['Year'][era]).text
        
        for person in root.findall(root_tags['People'][era]):
            dic = OrderedDict()
            dic['Year'] = year
            
            for key, tag in person_tags.items():
                value = person.find(tag[era])
                if value is not None:
                    value = value.text
                dic[key] = value
        
            list0.append(dic)
        
# display(list0)

In [ ]:
df0 = pd.DataFrame(list0)
df0['Year'] = df0['Year'].astype('int64')
cols = ['Name', 'Title']
df0[cols] = df0[cols].apply(lambda x: x.str.upper())
cols = ['Director', 'Officer', 'Highest', 'Former']
df0[cols] = df0[cols].replace({'X': True, None: False})
cols = ['Pay1', 'Pay2', 'Pay3']
df0[cols] = df0[cols].astype('int64')
cols = ['Hours1', 'Hours2']
df0[cols] = df0[cols].fillna(0).astype('float64')

display(df0)

In [ ]:
df1 = df0.copy()

# Drop these titles.
df1['Name'] = df1['Name'].str.replace('DR ', '')
df1['Name'] = df1['Name'].str.replace('BRIG ', '')

# Drop these suffixes.
df1['Name'] = df1['Name'].str.replace(' BS', '')
df1['Name'] = df1['Name'].str.replace(' JD', '')
df1['Name'] = df1['Name'].str.replace(' PHD', '')
df1['Name'] = df1['Name'].str.replace(' SM', '')
df1['Name'] = df1['Name'].str.replace(' KM', '')

# Rename.
df1['Name'] = df1['Name'].str.replace('GENERAL ', 'GEN ')
df1['Name'] = df1['Name'].str.replace('HONORABLE ', 'HON ')

# Deal with some special cases.
df1['Name'] = df1['Name'].replace({
    'AULANI KAANOI':
    'AYDEEN AULANI KAANOI',
    
    'DIANE PETERS NGUYEN':
    'DIANE PETERS-NGUYEN',
    
    'GAE BERGQUIST TROMMALD':
    'GAE BERGQUIST-TROMMALD',
    
    # Use name on Chaminade website.
    'CAROLYN A WILSON BERRY':
    'CAROLYN BERRY WILSON',

    'CAROLYN BERRY':
    'CAROLYN BERRY WILSON',
    
    # One person???
    'BENNETTE M EVANGELISTA':
    'BENNETTE E MISALUCHA',
    
    'BENNETTE M MISALUCHA':
    'BENNETTE E MISALUCHA',
    
    'BENNETTE MISSALUCHA':
    'BENNETTE E MISALUCHA'})

# These are the same people, probably.
df1['Name'] = df1['Name'].replace({
    'JERRY J CORREA':
    'JERRY J CORREA JR',
    
    'JOHN D FIELD':
    'JOHN D FIELD JR'})

# We add the middle initial and title if it's missing in places.
df1['Name'] = df1['Name'].replace({
    'HOYT ZIA':
    'HOYT H ZIA',
    
    'JEAN ROLLES':
    'JEAN E ROLLES',

    'LANCE MIZUMOTO':
    'LANCE A MIZUMOTO',

    'LAURIE TOM':   
    'LAURIE KS TOM',    
    
    'LAWRENCE TSEU':
    'LAWRENCE KW TSEU',

    'MICHAEL KERR':
    'MICHAEL F KERR',

    'PATRICK KSL YIM':    
    'HON PATRICK KSL YIM',
    
    'RICHARD KIDO':
    'RICHARD Y KIDO',

    'SCOTT SCHROEDER':
    'SCOTT J SCHROEDER',

    'SHELLEY WILSON':
    'SHELLEY J WILSON',

    'VAUGHN VASCONCELLOS':  
    'VAUGHN GA VASCONCELLOS'})

In [ ]:
# import nameparser
# constants = nameparser.config.Constants()
# constants.titles.add('BRO');
# constants.suffix_acronyms.add('BS', 'SM', 'KM')

# list1 = []
# for text in np.unique(df1['Name']):
#     name = nameparser.HumanName(text, constants=constants)
#     list1 += [f'{name.last}, {name.first}: {text}']
    
# list1.sort()
# display(list1)

In [ ]:
logic1 = df1['Officer'] | df1['Highest']
logic2 = df1['Hours1'] > 30
logic3 = logic1 & logic2
cols = ['Year', 'Name', 'Officer', 'Highest', 'Pay1']
df2 = df1.loc[logic3, cols].reset_index(drop=True)

display(df2)

In [ ]:
df3 = df2.pivot(index='Name', columns='Year', values='Pay1')
names = df3.max(axis=1).sort_values(ascending=False).index
df3 = df3.loc[names]

df3 = df3.applymap(lambda x: f'${x:,.0f}').replace('$nan', '-')
df3.columns.name = None
df3.index.name = None
df3.index = df3.index.map(lambda x: x.title())

display(df3)